In [1]:
import torch
import pandas as pd
from datetime import timedelta
from datetime import datetime

import numpy as np

from nera.data import *
from nera.models.ratings import EloAnalytical, EloManual, EloNumerical
from nera.reference import *
from nera.trainer import Trainer
from nera.utils import print_rating_diff


/home/kubos/anaconda3/envs/bp/lib/python3.11/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
da = DataAcquisition()
df = da.get_data(FROM_CSV, fname="../resources/other_leagues.csv")
df['DT'] = pd.to_datetime(df['DT'], format="%Y-%m-%d %H:%M:%S")
#df = df[(df['League'] != 'EuroLeague') & (df['League'] != 'EuroCup')] 
df = df.reset_index()
df = df.sort_values(by='DT', ascending=False)

transform = DataTransformation(df, timedelta(365))
dataset = transform.get_dataset(node_f_extract=False, edge_f_one_hot=True)

team_count = transform.num_teams

print(len(df))

2024-03-04 15:21:10.555 | INFO     | nera.data._data_saving_loading:load_data_csv:70 - 21100 rows loaded from ../resources/other_leagues.csv


21100


### Manual Elo with NN pass

In [3]:
# dummy dataset

delta = timedelta(seconds=1)
delta2 = timedelta(days=366)
now = datetime.now()
data = pd.DataFrame({'DT': [*[now - i * delta2 + j * delta for j in range(3) for i in range(4)]], 
                     'Home': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
                     'Away': ['B', 'C', 'A', 'C', 'A', 'B', 'B', 'C', 'A', 'C', 'A', 'B'],
                     'Winner': ['home', 'away', 'away', 'home', 'away', 'away', 'home', 'home', 'away', 'home', 'away', 'home'],
                     'Home_points': [10, 5, 15, 12, 15, 6, 20, 10, 10, 14, 3, 12],
                     'Away_points': [4, 8, 17, 10, 16, 14, 18, 9, 15, 0, 11, 4],
                     'League': [*(12 * ['liga'])],
                     })
data = data.sort_values(by='DT', ascending=False)
transform2 = DataTransformation(data, timedelta(days=365))
dummy_dataset = transform2.get_dataset(edge_f_one_hot=True)

In [4]:
elo_man = EloManual(team_count=transform2.num_teams)
trainer = Trainer(dummy_dataset, train_ratio=1)
trainer.model = elo_man
reference_maker = RatingReference(transform2.num_teams)

print('Computed elo Dummy: ', reference_maker.compute_reference('elo', dummy_dataset)[0])
acc_dummy = trainer.train(epochs=1, val_ratio=0, verbose=True)
print(elo_man.elo)
print()
print('----------------------------------------')
print()

reference_maker.num_teams = transform.num_teams
computed = reference_maker.compute_reference('elo', dataset)[0]

print()
elo = EloManual(team_count=transform.num_teams)
trainer.dataset = dataset
trainer.model = elo
acc_late = trainer.train(epochs=1, val_ratio=0, verbose=True)

err = False
for i in range(len(computed)):
    cmp = float(computed[i])
    net = float(elo.elo[i])
    if abs(cmp - net) > 0.1:
        print(rf'ERROR on index {i}:: {cmp} / {net}')
        err = True
    if i < 5:
        print(f'{i}:: computed: {cmp:10.3f} || net: {net:10.3f}')
if not err:
    print("...")
    print("[SUCCESS]: Computed elo is the same as Elo from NN")



2024-03-04 15:21:10.954 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 174.165, training accuracy: 75.00%


Computed elo Dummy:  [ 658.38739014  880.46270752 1461.15014648]
Parameter containing:
tensor([ 658.3874,  880.4627, 1461.1500], dtype=torch.float64,
       requires_grad=True)

----------------------------------------


2024-03-04 15:21:18.114 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 2293848.884, training accuracy: 59.09%


0:: computed:   7191.367 || net:   7191.367
1:: computed:   2895.177 || net:   2895.178
2:: computed:   3006.513 || net:   3006.513
3:: computed:   4540.632 || net:   4540.633
4:: computed:   3065.280 || net:   3065.280
...
[SUCCESS]: Computed elo is the same as Elo from NN


### Elo with gradient

first lets check our analytical backward pass

In [12]:
from torch.autograd import gradcheck
from nera.models.ratings._elo._analytical import elo_function
# gradcheck takes a tuple of tensors as input, check if your gradient
# evaluated with these tensors are close enough to numerical
# approximations and returns True if they all verify this condition.
input = (torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True), torch.randn(1,dtype=torch.double,requires_grad=True))
test = gradcheck(elo_function, input, eps=1e-6, atol=1e-4)
print(test)

True


Define training function for elo

Now, let's train elo with analytical backward pass

In [13]:
_elo_params = {
    'gamma': 0.6,
    'c': 3.,
    'd': 500.,
}
trainer.set_lr(1, 'rating')
elo_grad = EloAnalytical(team_count, hp_grad=False, **_elo_params)
trainer.model = elo_grad
trainer.train_ratio = 0.8
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
trainer.test(verbose=True)

2024-03-04 15:21:58.570 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 19778.735, training accuracy: 49.12%
2024-03-04 15:21:58.572 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 5272.238, validation accuracy: 50.03%
2024-03-04 15:21:58.574 | INFO     | nera.trainer:test:267 - [TST] Testing accuracy: 45.83%


0.4583333333333333

And Elo with autograd (numerical gradient backward pass)

In [14]:
elo_auto = EloNumerical(team_count, hp_grad=False, **_elo_params)
trainer.model = elo_auto
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
trainer.test(verbose=True)

2024-03-04 15:22:07.543 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 19778.735, training accuracy: 49.12%
2024-03-04 15:22:07.545 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 5272.238, validation accuracy: 50.03%
2024-03-04 15:22:07.546 | INFO     | nera.trainer:test:267 - [TST] Testing accuracy: 45.83%


0.4583333333333333

And finally, compare wheter these two are the same
___________________

In [15]:
print_rating_diff(elo_auto, elo_grad, transform)

Ratings diff:
Rating 0:
0:: computed:    129.308 || net:    129.308
1:: computed:    323.576 || net:    323.576
2:: computed:   1159.981 || net:   1159.981
3:: computed:   1269.019 || net:   1269.019
4:: computed:   1721.490 || net:   1721.491
...

[SUCCESS]: All  ratings computed analytically and numerically are the SAME
-------------------------

Hyperparams diff:


Now let's see what happens, when we let the model learn the c, d metaparameters as well

In [34]:
elo_auto2 = EloNumerical(team_count, hp_grad=True, c=torch.tensor(3, dtype=torch.float64), d=torch.tensor(500, dtype=torch.float64))
trainer.model = elo_auto2
trainer.set_lr(0.00001)
trainer.set_lr(1, 'rating')
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
trainer.test(verbose=True)

2024-03-04 15:27:10.482 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 1245772.886, training accuracy: 48.36%
2024-03-04 15:27:10.484 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 344329.175, validation accuracy: 47.32%
2024-03-04 15:27:10.486 | INFO     | nera.trainer:test:267 - [TST] Testing accuracy: 51.25%


0.5125

In [35]:
elo_grad2 = EloAnalytical(team_count, hp_grad=True, c=torch.tensor(3, dtype=torch.float64), d=torch.tensor(500, dtype=torch.float64))
trainer.model = elo_grad2
trainer.train(verbose=True, epochs=1, val_ratio=0.2)
trainer.test(verbose=True)

2024-03-04 15:27:22.134 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 1245772.888, training accuracy: 48.36%
2024-03-04 15:27:22.135 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 344329.176, validation accuracy: 47.32%
2024-03-04 15:27:22.137 | INFO     | nera.trainer:test:267 - [TST] Testing accuracy: 51.25%


0.5125

In [36]:
print_rating_diff(elo_auto2, elo_grad2, transform)

Ratings diff:
Rating 0:
0:: computed:    181.344 || net:    181.344
1:: computed:   -391.693 || net:   -391.694
2:: computed:   1156.948 || net:   1156.948
3:: computed:    524.137 || net:    524.137
4:: computed:   1458.303 || net:   1458.303
...

[SUCCESS]: All  ratings computed analytically and numerically are the SAME
-------------------------

Hyperparams diff:
Numerical: hyperparam[0] =    2.851 :: Numerical: hyperparam[0] =    2.851
Numerical: hyperparam[1] =  500.148 :: Numerical: hyperparam[1] =  500.148


In [37]:
elo = EloManual(team_count=transform.num_teams)
trainer.dataset = dataset
trainer.model = elo
acc_late = trainer.train(epochs=1, val_ratio=0.2, verbose=True)
trainer.test(verbose=True)

2024-03-04 15:27:26.578 | INFO     | nera.trainer:train:171 - [TRN] Epoch: 0, training loss: 1620577.841, training accuracy: 59.20%
2024-03-04 15:27:26.579 | INFO     | nera.trainer:train:174 - [VAL] Epoch: 0, validation loss: 424847.971, validation accuracy: 57.11%
2024-03-04 15:27:26.581 | INFO     | nera.trainer:test:267 - [TST] Testing accuracy: 56.25%


0.5625